  - Realice una clasificación de los mismos y evalúe su desempeño utilizando la métrica de evaluación AMS.

In [3]:
import numpy as np
import pandas as pd
import pickle

In [4]:
import sys
sys.path.append("../../utils")

from HiggsBosonUtils import AMS

In [5]:
INPUT_MODELS_PATH = "./trained-models"
INPUT_DATA_PATH = "../../data/output"
data_training = pd.read_csv('../../data/input/training.csv')
X_train = np.loadtxt(f"{INPUT_DATA_PATH}/X_train.txt")
y_train = np.loadtxt(f"{INPUT_DATA_PATH}/y_train.txt")

weights = np.array(data_training['Weight'])

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [7]:
rf_clf = pickle.load(open(f"{INPUT_MODELS_PATH}/rf.sav", 'rb'))
dt_clf = pickle.load(open(f"{INPUT_MODELS_PATH}/dt.sav", 'rb'))

In [8]:
y_pred_dt = dt_clf.predict(X_train)

s = data_training.loc[(y_pred_dt == 1) & (y_train == 1)].Weight.sum()
b = data_training.loc[(y_pred_dt == 1) & (y_train == 0)].Weight.sum()

print('AMS train (Desicion Tree) = ', AMS(s, b))

AMS train (Desicion Tree) =  67.71112289505676


In [9]:
y_pred_rf = rf_clf.predict(X_train)

s = data_training.loc[(y_pred_rf == 1) & (y_train == 1)].Weight.sum()
b = data_training.loc[(y_pred_rf == 1) & (y_train == 0)].Weight.sum()

print('AMS train (Random Forest) = ', AMS(s, b))

AMS train (Random Forest) =  66.39744963507958


In [10]:
s = data_training.loc[ (y_train == 1)].Weight.sum()
b = 0

print('AMS train (Caso Ideal) = ', AMS(s, b))

AMS train (Caso Ideal) =  67.71112289505676


In [11]:
s = data_training.loc[y_train == 1].Weight.sum()
b = data_training.loc[y_train == 0].Weight.sum()

print('AMS train (todos se predicen como Signal) = ', AMS(s, b))

AMS train (todos se predicen como Signal) =  1.0790735173438217


In [12]:
s = 0
b = 0

print('AMS train (todos se predicen como Background) = ', AMS(s, b))

AMS train (todos se predicen como Background) =  0.0


Peor caso AMS = 0, pq s=0 (train)

 ## Parte 3 -  Un primer pipeline

 - Probar clasificar los datos del proyecto con los distintos métodos vistos en este taller (Árboles de Decisión, Random Forest, XGBoost).

In [13]:
dt_clf = pickle.load(open(f"{INPUT_MODELS_PATH}/dt.sav", 'rb'))

In [14]:
y_pred_dt = dt_clf.predict(X_train)

s = data_training.loc[(y_pred_dt == 1) & (y_train == 1)].Weight.sum()
b = data_training.loc[(y_pred_dt == 1) & (y_train == 0)].Weight.sum()

print('AMS train (desicion tree)= ', AMS(s, b))

AMS train (desicion tree)=  67.71112289505676


In [15]:
xgb_clf = pickle.load(open(f"{INPUT_MODELS_PATH}/xgb.sav", 'rb'))

In [16]:
y_pred_xgb = xgb_clf.predict(X_train)

s = data_training.loc[(y_pred_xgb == 1) & (y_train == 1)].Weight.sum()
b = data_training.loc[(y_pred_xgb == 1) & (y_train == 0)].Weight.sum()

print('AMS train (XGBoost) = ', AMS(s, b))

AMS train (XGBoost) =  3.0632911355363444


 ## Parte 4 -  Un primer pipeline
Descargue la implementación de la métrica de la página del [curso](https://eva.fing.edu.uy/mod/resource/view.php?id=135850). Modifique el nombre del archivo descargado a *HiggsBosonUtils.py* y guarde dicho archivo en una carpeta *tools* en el mismo directorio donde está el presente Notebook. En estas condiciones, puede importar la función *AMS* contenida dentro de *HiggsBosonUtils.py* de la siguiente forma:  

- será necesario también fijar un punto de operación que maximice el desempeño de la métrica, en este caso, el *AMS*. En función de ello, resulta pertinente observar que valores resultan razonables para dicha métrica.

#### Objetivos:

 - Levantar el conjunto de datos del proyecto. 
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica todos los eventos como *background*.
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica todos los eventos como *signal*.
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica perfectamente todos los eventos.
 - Observe los Leaderboard de Kaggle y en base a los resultados obtenidos anteriormente que discuta que valores le resultan razonables de *AMS*.
 - Con los datos del proyecto genere dos conjuntos uno de Entrenamiento y otro de Validación. Luego, entrene el modelo y evalúe el desempeño en el conjunto de Validación. Observe como varía el AMS al modificar el umbral.

In [23]:
from evaluate_models import AMSGridSearchCV

print(AMSGridSearchCV(RandomForestClassifier(), {'n_estimators': [50, 200]}, 3, X_train, y_train, weights))

[2.798954573868768]
[2.798954573868768, 2.8007368524373266]
[2.798954573868768, 2.8007368524373266, 2.783429848605857]
[2.8541866262279934]
[2.8541866262279934, 2.8485576797445167]
[2.8541866262279934, 2.8485576797445167, 2.851815047121342]
([2.8541866262279934, 2.8485576797445167, 2.851815047121342], {'n_estimators': 200})
